In [1]:
from selenium import webdriver
import time
import requests

url = 'http://corners.gmarket.co.kr/Bestsellers'
driver = webdriver.Chrome("C:/Users/minO/chromedriver/chromedriver.exe")
driver.get(url)

In [51]:
items = driver.find_elements_by_css_selector(
 ".best-list:nth-child(3) > ul > li")
len(items)

200

In [52]:
dict_list = []
for item in items:
    title = item.find_element_by_css_selector(".itemname").text
    link = item.find_element_by_css_selector(".itemname").get_attribute("href")
    o_price = item.find_element_by_css_selector(".o-price").text[:-1]
    s_price = item.find_element_by_css_selector(".s-price").text.split(" ")[0][:-1]
    
    data = {
        "title":title,
        "link":link,
        "o_price":o_price,
        "s_price":s_price,
    }
    dict_list.append(data)

In [53]:
driver.quit()

In [54]:
import pandas as pd
df = pd.DataFrame(dict_list)
df.head()

,title,link,o_price,s_price
0,[제스프리](M)제스프리 썬골드키위 점보사이즈 3.1kg(과당 156g 내외 19-...,http://item.gmarket.co.kr/Item?goodscode=21321...,"30,900","28,500"
1,달콤아삭 특품 샤인머스켓 2kg (3수~4수),http://item.gmarket.co.kr/Item?goodscode=21925...,"33,900","33,570"
2,[제스프리](M)제스프리 썬골드키위 특대사이즈 3.1kg(과당 120g-140g 내...,http://item.gmarket.co.kr/Item?goodscode=20999...,"30,900","28,500"
3,한양식품 꽃보다오징어 오리지날 260g+260g 외,http://item.gmarket.co.kr/Item?goodscode=21982...,,"32,900"
4,(깨비농원)고당도 하우스귤 4.5kg 소과,http://item.gmarket.co.kr/Item?goodscode=16164...,"34,900","33,510"


In [55]:
nodata_indexs = df["o_price"] == ""

In [57]:
df["o_price"][nodata_indexs].tail()

185    
189    
191    
192    
195    
Name: o_price, dtype: object

In [58]:
df["s_price"][nodata_indexs].tail()

185     56,500
189      5,990
191     30,900
192    149,000
195     19,900
Name: s_price, dtype: object

In [59]:
df[nodata_indexs].tail()

,title,link,o_price,s_price
185,윤쌀 10kg+10kg 쌀 20년산 (박스 포장),http://item.gmarket.co.kr/Item?goodscode=18505...,,"56,500"
189,(전단상품)온가족닭볶음탕용 1.2kg 팩,http://item.gmarket.co.kr/Item?goodscode=20446...,,"5,990"
191,비비고 만두 총 10팩(왕교자X2+김치왕교자X2+매운왕교자X2+고기만두X2+김치만,http://item.gmarket.co.kr/Item?goodscode=21251...,,"30,900"
192,[삼성전자]7%할인 갤럭시버즈2+케이스+CU8천원+경품EVENT,http://item.gmarket.co.kr/Item?goodscode=21898...,,"149,000"
195,향긋한 여름 털복숭아 4.5kg/20과내외,http://item.gmarket.co.kr/Item?goodscode=21607...,,"19,900"


In [60]:
df["o_price"][nodata_indexs] = df["s_price"][nodata_indexs]

In [63]:
df[nodata_indexs].head()

,title,link,o_price,s_price
3,한양식품 꽃보다오징어 오리지날 260g+260g 외,http://item.gmarket.co.kr/Item?goodscode=21982...,"32,900","32,900"
5,두메산골 냉장 生닭다리살 (정육) 1kg+1kg(총2kg),http://item.gmarket.co.kr/Item?goodscode=21106...,"23,900","23,900"
9,[참치브라더]노르웨이 슈페리얼 생연어 블럭 1kg,http://item.gmarket.co.kr/Item?goodscode=17249...,"27,900","27,900"
11,양희경 서울식 포기김치 10Kg/국내산 쿠폰적용 22120원,http://item.gmarket.co.kr/Item?goodscode=12783...,"31,900","31,900"
12,[산과들에]하루 한줌 견과 원데이프리미엄100낱봉 +10낱봉+사은품,http://item.gmarket.co.kr/Item?goodscode=15778...,"41,500","41,500"


In [64]:
df.tail()

,title,link,o_price,s_price
195,향긋한 여름 털복숭아 4.5kg/20과내외,http://item.gmarket.co.kr/Item?goodscode=21607...,"19,900","19,900"
196,[베베앙]베베앙 아기물티슈 시그니처 대용량 100매 캡형 10팩,http://item.gmarket.co.kr/Item?goodscode=16044...,"23,800","11,900"
197,[페리오]어린이칫솔 쿠폰가13530원 2단계핑크퐁칫솔20입+사은품,http://item.gmarket.co.kr/Item?goodscode=17038...,"20,100","17,900"
198,수퍼맘스토리카페/재밌고 기억하기 쉽게 가르쳐주는/미국초등수학교과서/맥그로힐/McGr...,http://item.gmarket.co.kr/Item?goodscode=21923...,"68,000","48,800"
199,[구글플레이](카드가능) 기프트코드 20만원 / 구글 기프트카드,http://item.gmarket.co.kr/Item?goodscode=16552...,"200,000","190,000"


In [67]:
df["o_price"] = df["o_price"].apply(
    lambda price: price.replace(",","")).astype("int")
df["s_price"] = df["s_price"].apply(
    lambda price: price.replace(",","")).astype("int")

In [68]:
df["sale_rate"] = round((1-df["s_price"]/df["o_price"])*100,2)

In [69]:
df.tail()

,title,link,o_price,s_price,sale_rate
195,향긋한 여름 털복숭아 4.5kg/20과내외,http://item.gmarket.co.kr/Item?goodscode=21607...,19900,19900,0.00
196,[베베앙]베베앙 아기물티슈 시그니처 대용량 100매 캡형 10팩,http://item.gmarket.co.kr/Item?goodscode=16044...,23800,11900,50.00
197,[페리오]어린이칫솔 쿠폰가13530원 2단계핑크퐁칫솔20입+사은품,http://item.gmarket.co.kr/Item?goodscode=17038...,20100,17900,10.95
198,수퍼맘스토리카페/재밌고 기억하기 쉽게 가르쳐주는/미국초등수학교과서/맥그로힐/McGr...,http://item.gmarket.co.kr/Item?goodscode=21923...,68000,48800,28.24
199,[구글플레이](카드가능) 기프트코드 20만원 / 구글 기프트카드,http://item.gmarket.co.kr/Item?goodscode=16552...,200000,190000,5.00


In [70]:
df[df["sale_rate"]>50].sort_values("sale_rate", ascending=False).reset_index(drop=True).head()

,title,link,o_price,s_price,sale_rate
0,엄마옷로즈맘 시즌오프세일 인견5060중년여성의류,http://item.gmarket.co.kr/Item?goodscode=77658...,43000,12900,70.0
1,[엄지한복]20%+15% 엄지한복 남아/여아 균일가+복주머니 증정,http://item.gmarket.co.kr/Item?goodscode=16013...,130000,39000,70.0
2,[벨레다]벨레다공식총판 벨레다 어린이치약 50ml 6개 +증정품,http://item.gmarket.co.kr/Item?goodscode=19039...,143330,43000,70.0
3,데이데이 가을블라우스/맨투맨/팬츠/니트/가디건/팬츠,http://item.gmarket.co.kr/Item?goodscode=15706...,33000,9900,70.0
4,[캔디베이비]70%할인 시즌오프 유아동내의/내복/실내복,http://item.gmarket.co.kr/Item?goodscode=43858...,19000,5700,70.0
